In [5]:
import torch
from torch import nn, Tensor
import transformers
from transformers import BertTokenizer, BertModel, BertConfig, EncoderDecoderModel, BertForMaskedLM

In [5]:
# bert2bert
bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [7]:
print(bert2bert)

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [8]:
encoder = BertModel.from_pretrained("bert-base-uncased") # config=encoder_config
decoder = BertForMaskedLM.from_pretrained("bert-base-uncased") #config=decoder_config
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relatio

In [12]:
print(model)

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [15]:
def count_parameters(model):
    # table = PrettyTable(["Modules", "Parameters"])
    table = {}
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table[name] = params
        total_params+=params
    # print(table)
    
    print(f"Total Trainable Params: {total_params}")
    return total_params, table
    
total_params, table = count_parameters(model)
for k, v in table.items():
    print(k, v)
    

Total Trainable Params: 218996538
encoder.embeddings.word_embeddings.weight 23440896
encoder.embeddings.position_embeddings.weight 393216
encoder.embeddings.token_type_embeddings.weight 1536
encoder.embeddings.LayerNorm.weight 768
encoder.embeddings.LayerNorm.bias 768
encoder.encoder.layer.0.attention.self.query.weight 589824
encoder.encoder.layer.0.attention.self.query.bias 768
encoder.encoder.layer.0.attention.self.key.weight 589824
encoder.encoder.layer.0.attention.self.key.bias 768
encoder.encoder.layer.0.attention.self.value.weight 589824
encoder.encoder.layer.0.attention.self.value.bias 768
encoder.encoder.layer.0.attention.output.dense.weight 589824
encoder.encoder.layer.0.attention.output.dense.bias 768
encoder.encoder.layer.0.attention.output.LayerNorm.weight 768
encoder.encoder.layer.0.attention.output.LayerNorm.bias 768
encoder.encoder.layer.0.intermediate.dense.weight 2359296
encoder.encoder.layer.0.intermediate.dense.bias 3072
encoder.encoder.layer.0.output.dense.weight 23

In [19]:
# freeze every thing but the last cls 
for name, parameter in model.named_parameters():
    if name[:11] == 'decoder.cls':
        print(name)

decoder.cls.predictions.bias
decoder.cls.predictions.transform.dense.weight
decoder.cls.predictions.transform.dense.bias
decoder.cls.predictions.transform.LayerNorm.weight
decoder.cls.predictions.transform.LayerNorm.bias
